## Load and prep the data for processing


In [36]:
import datetime
import pandas as pd
from pathlib import Path
from urllib.parse import parse_qs
from dateutil import relativedelta

# prep the environment
data_dir = Path.cwd() / "data"
# uat_file_name = "ssrs_week_1_2.csv"
uat_file_name = "ssrs_week_1.csv"
uat_file = data_dir / uat_file_name
data_file = data_dir / "uat_data.pkl"
uat_start = datetime.datetime(2019, 7, 22)


In [24]:
# load the UAT runtime data
if uat_file.exists():
    raw_df = pd.read_csv(uat_file)
else:
    print(f"Processed data missing, please run the Data Prep notebook first.")
    exit(-1)


### Select Key Columns:
- All (for now)


In [25]:
# select key columns
uat_df = raw_df


### Format Columns:
1. DateTime columns = StartTime, EndTime


In [26]:
uat_df['StartTime'] = pd.to_datetime(uat_df['StartTime'], infer_datetime_format=True)
uat_df['StopTime'] = pd.to_datetime(uat_df['StopTime'], infer_datetime_format=True)


### Clean Invalid Rows:
1. Rows with NULL in the following columns:
  - User
  - ReportPath
1. Rows with StartTime before the start of UAT

  

In [27]:
# clean invalid the rows
uat_df.dropna(subset=['User', 'ReportPath'], inplace=True)

# clean pre-UAT rows
uat_df = uat_df[uat_df['StartTime'] >= uat_start]


### Create Derived Fields:
1. DurationTotal - the sum of the 3 duration columns
1. DurationTotalSec - converting DurationTotal to seconds
1. DurationDataRetrievalSec - converting DurationDataRetrieval to seconds
1. DurationProcessingSec - converting DurationProcessing to seconds
1. DurationRenderingSec - converting DurationRendering to seconds
1. ReportGroup - the broad group of users (directory)
1. ReportName - the name portion of the report
1. Agency - the Agency of the user who ran the report
1. Dims - report on how many times report execution called a /*Dim record 
   (then remove the /*Dim rows from the DataFrame)


In [28]:
# create derived fields
uat_df['DurationTotal'] = uat_df['DurationDataRetrieval'] + uat_df['DurationProcessing'] + uat_df['DurationRendering']
uat_df['DurationTotalSec'] = uat_df['DurationTotal'] / 1000
uat_df['DurationDataRetrievalSec'] = uat_df['DurationDataRetrieval'] / 1000
uat_df['DurationProcessingSec'] = uat_df['DurationProcessing'] / 1000
uat_df['DurationRenderingSec'] = uat_df['DurationRendering'] / 1000
uat_df[['ReportGroup', 'ReportName']] = uat_df['ReportPath'].str.extract('^/(?P<UserGroup>.*)/(?P<ReportName>.*$)')[['UserGroup', 'ReportName']]

# set field types
uat_df['ReportGroup'] = uat_df['ReportGroup'].astype('category')
uat_df['ReportName'] = uat_df['ReportName'].astype('category')

# Add Agency
uat_df.User.replace('Administrator', 'UATUserDARe', inplace=True)
uat_df.User.replace('UATUserOrca', 'UATUserORCA', inplace=True)
uat_df['Agency'] = uat_df['User'].str.extract('.*UATUser(?P<Agency>.*)$')

# Filter out Dim records
dim_list = ['/AgencyDim', '/BranchDim', '/CardGroupDim','/InstitutionDim', '/ParticipantDim']
dim_df = uat_df[uat_df.ReportPath.isin(dim_list)]
uat_df = uat_df[~uat_df.ReportPath.isin(dim_list)]
print(f"Filtered out {len(dim_df)} /*Dim records, making an average of {(len(dim_df) / len(uat_df)):.2f} /*Dim calls per ExecutionId")


Filtered out 1267 /*Dim records, making an average of 2.04 /*Dim calls per ExecutionId


/Users/mcclure/PycharmProjects/dare_analytics/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mcclure/PycharmProjects/dare_analytics/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mcclure/PycharmProjects/dare_analytics/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A va

### Process the Query Parameters

In [38]:
param_num_institutions = lambda x: len(parse_qs(x)['InstitutionDimKey']) if type(x) == str and 'InstitutionDimKey' in parse_qs(x).keys() else 0

def param_query_months(param_string):
    date_format = '%m/%d/%Y %I:%M:%S %p'
    if type(param_string) != str:
        return None
    params = parse_qs(param_string)
    if 'FromDate' not in params.keys() or 'ToDate' not in params.keys():
        return None
    rdelta = relativedelta.relativedelta(datetime.datetime.strptime(params['ToDate'][0], date_format),
                                         datetime.datetime.strptime(params['FromDate'][0], date_format))
    if rdelta.days > 16:
        rdelta.months += 1
    months = rdelta.years * 12 + rdelta.months
    return months

uat_df['ParamNumInstitutions'] = uat_df['Parameters'].apply(param_num_institutions)
uat_df['ParamNumMonths'] = uat_df['Parameters'].apply(param_query_months)

### Save the processed DataFrame

In [39]:
# save the loaded data
uat_df.to_pickle(data_file)
print(f"Updated {data_file} on {datetime.datetime.now()}")

Updated /Users/mcclure/PycharmProjects/dare_analytics/data/uat_data.pkl on 2019-08-07 17:03:03.553195
